정규화를 해서 dt에 넣어보기 -> 모든 데이터를 0~1 사이의 값이 될 수 있또록
lead_time, avg_price_per_room -> cate ->  num으로 바꾸기


In [113]:
!pip install -U imbalanced-learn


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [111]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

In [114]:

train = pd.read_csv('final_train.csv')
train.keys()

Index(['Booking_ID', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'booking_status',
       'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
       'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
       'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
       'Complementary', 'Online', 'Offline', 'Aviation',
       'lead_time_under_iqr_1', 'lead_time_under_iqr_2',
       'lead_time_under_iqr_3', 'lead_time_under_iqr_4', 'price_under_iqr_1',
       'price_under_iqr_2', 'price_under_iqr_3', 'price_under_iqr_4',
       'lead_time_norm', 'price_norm'],
      dtype='object')

In [ ]:
X = train[feature_names]
y = train["booking_status"]
smote = SMOTEENN(random_state = 0)
X_train_over, y_train_over = smote.fit_resample(X, y)

In [165]:
'''
feature_names = ['no_of_adults', 'no_of_children',
'no_of_weekend_nights',
                 'no_of_week_nights', 'required_car_parking_space',
                 'lead_time', 'arrival_year', 'arrival_month',
                 'arrival_date', 'repeated_guest',
                 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
                 'avg_price_per_room', 'no_of_special_requests',
                 'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
                 'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
                 'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
                 'Complementary', 'Online', 'Offline', 'Aviation']
'''

train = pd.read_csv('final_train.csv')
train['lead_12'] = train['lead_time_under_iqr_1'] + train['lead_time_under_iqr_2']
train['lead_34'] = train['lead_time_under_iqr_3'] + train['lead_time_under_iqr_4']
train['price_12'] = train['price_under_iqr_1'] + train['price_under_iqr_2']
train['price_34'] = train['price_under_iqr_3'] + train['price_under_iqr_4']

feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'required_car_parking_space',
       'arrival_year', 'arrival_month',
       'arrival_date', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'no_of_special_requests', 
       'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
       'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
       'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
       'Complementary', 'Online', 'Offline', 'Aviation',
       'lead_time_norm', 'price_norm', 'lead_12', 'lead_34', 'price_12', 'price_34']

# 87 프로 피쳐
# train = pd.read_csv('onehot_sample.csv')

# feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
#                  'no_of_week_nights', 'required_car_parking_space',
#                  'lead_time', 'arrival_year', 'arrival_month',
#                  'arrival_date', 'repeated_guest',
#                  'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
#                  'avg_price_per_room', 'no_of_special_requests',
#                  'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
#                  'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
#                  'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
#                  'Complementary', 'Online', 'Offline', 'Aviation']

X = train[feature_names]
y = train["booking_status"]
smote = SMOTE(random_state = 42)
# Decision Tree 학습/예측/평가
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)

# Decision Tree 학습/예측/평가
dt_model = DecisionTreeClassifier(max_depth = 15, min_samples_leaf = 2, min_samples_split = 2, criterion="gini", max_features = 25)
scores = cross_val_score(dt_model, X, y, cv = 50)
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88779528 0.87598425 0.86614173 0.85629921 0.87992126 0.85826772
 0.87598425 0.88385827 0.89370079 0.88188976 0.86023622 0.86614173
 0.88385827 0.87598425 0.89173228 0.90551181 0.84055118 0.87401575
 0.86811024 0.88582677 0.86023622 0.86417323 0.88385827 0.88582677
 0.87007874 0.8503937  0.89173228 0.87204724 0.88976378 0.87401575
 0.87795276 0.87992126 0.85629921 0.86220472 0.87204724 0.9015748
 0.88385827 0.85433071 0.87795276 0.86811024 0.88385827 0.88385827
 0.86193294 0.89940828 0.86193294 0.87179487 0.85601578 0.85996055
 0.85404339 0.8816568 ]
Average CV Score:  0.8740530214788241
Number of CV Scores used in Average:  50


In [155]:
def get_dt_graph(dt_classifier):
    fig = plt.figure(figsize=(25, 20))
    _ = dt_model.plot_tree(dt_classifier,
                       feature_names=X.columns,
                       class_names=['No Disease', "Disease"],
                       filled=True)


def evaluate_model(dt_classifier):
    print("Train Accuracy :", accuracy_score(
        y_train, dt_classifier.predict(X_train)))
    print("Train Confusion Matrix:")
    print(confusion_matrix(y_train, dt_classifier.predict(X_train)))
    print("-"*50)
    print("Test Accuracy :", accuracy_score(
        y_test, dt_classifier.predict(X_test)))
    print("Test Confusion Matrix:")
    print(confusion_matrix(y_test, dt_classifier.predict(X_test)))


params = {
    'max_depth' : [8, 10, 15, 20], 
    'min_samples_leaf' : [2, 4, 6, 8], 
    'min_samples_split' : [2, 4, 6, 8],
    'max_features' : [25, 30, 25, 40]
}

# params = {
#     'max_features' : [5, 10, 15, 20, 25, 30, 35]
# }

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=dt_model,
                           param_grid=params,
                           cv=10, n_jobs=-1, verbose=1, scoring="accuracy")

grid_search.fit(X_train, y_train)


Fitting 10 folds for each of 256 candidates, totalling 2560 fits


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
640 fits failed out of a total of 2560.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
640 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py", 

GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(max_depth=15, max_features=30,
                                              min_samples_leaf=2),
             n_jobs=-1,
             param_grid={'max_depth': [8, 10, 15, 20],
                         'max_features': [25, 30, 25, 40],
                         'min_samples_leaf': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6, 8]},
             scoring='accuracy', verbose=1)

In [156]:
GridSearchCV(cv = 10, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_samples_split': [5, 10, 20, 50, 100]},
             scoring='accuracy', verbose=1)

score_df = pd.DataFrame(grid_search.cv_results_)
# score_df.to_csv('dt_model_grid_search.csv')
score_df.nlargest(5, "mean_test_score")


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
144,0.152400,0.029406,0.004989,0.002522,15,30,2,2,"{'max_depth': 15, 'max_features': 30, 'min_sam...",0.849344,...,0.864567,0.858718,0.876050,0.873424,0.865546,0.882878,0.873424,0.869513,0.010067,1
130,0.126103,0.033440,0.005540,0.004203,15,25,2,6,"{'max_depth': 15, 'max_features': 25, 'min_sam...",0.845144,...,0.862467,0.870273,0.879202,0.872899,0.870273,0.865021,0.871849,0.869198,0.010974,2
145,0.117539,0.010522,0.003591,0.000661,15,30,2,4,"{'max_depth': 15, 'max_features': 30, 'min_sam...",0.844094,...,0.865092,0.855567,0.870273,0.876050,0.873424,0.878151,0.870798,0.868988,0.011059,3
161,0.125506,0.030567,0.004484,0.001048,15,25,2,4,"{'max_depth': 15, 'max_features': 25, 'min_sam...",0.844619,...,0.871916,0.856618,0.871849,0.876050,0.867647,0.876576,0.868172,0.868830,0.010085,4
128,0.110737,0.022340,0.004388,0.000798,15,25,2,2,"{'max_depth': 15, 'max_features': 25, 'min_sam...",0.846719,...,0.874541,0.867647,0.863971,0.868697,0.878151,0.866071,0.872899,0.868673,0.008433,5


In [166]:

dt_model.fit(X_train_over, y_train_over)


DecisionTreeClassifier(max_depth=15, max_features=25, min_samples_leaf=2,
                       min_samples_split=4)

In [167]:
y_pred = dt_model.predict(X_test)
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))
evaluate_model(dt_model)

Accuracy: 0.86
Train Accuracy : 0.9133585381222432
Train Confusion Matrix:
[[11975   860]
 [  790  5419]]
--------------------------------------------------
Test Accuracy : 0.8649968494013862
Test Confusion Matrix:
[[3807  472]
 [ 385 1684]]


In [99]:
from sklearn.preprocessing import OneHotEncoder

real_test = pd.read_csv('test.csv')

ohe = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
train_cat = ohe.fit_transform(real_test[['type_of_meal_plan']])
train_cat = pd.DataFrame(train_cat)
real_test['meal_type_1'] = train_cat.loc[:, 0]
real_test['meal_type_2'] = train_cat.loc[:, 1]
real_test['meal_type_3'] = train_cat.loc[:, 2]
real_test['meal_type_4'] = train_cat.loc[:, 3]

ohe = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
train_cat = ohe.fit_transform(real_test[['room_type_reserved']])
train_cat = pd.DataFrame(train_cat)
real_test['Room_Type_1'] = train_cat.loc[:, 0]
real_test['Room_Type_2'] = train_cat.loc[:, 1]
real_test['Room_Type_3'] = train_cat.loc[:, 2]
real_test['Room_Type_4'] = train_cat.loc[:, 3]
real_test['Room_Type_5'] = train_cat.loc[:, 4]
real_test['Room_Type_6'] = train_cat.loc[:, 5]
real_test['Room_Type_7'] = train_cat.loc[:, 6]

train_cat = ohe.fit_transform(real_test[['market_segment_type']])
train_cat = pd.DataFrame(train_cat)
real_test['Corporate'] = train_cat.loc[:, 0]
real_test['Complementary'] = train_cat.loc[:, 1]
real_test['Online'] = train_cat.loc[:, 2]
real_test['Offline'] = train_cat.loc[:, 3]
real_test['Aviation'] = train_cat.loc[:, 4]

lead_time_norm = (real_test['lead_time'] - real_test['lead_time'].mean())/real_test['lead_time'].std()
price_norm = (real_test['avg_price_per_room'] - real_test['avg_price_per_room'].mean())/real_test['avg_price_per_room'].std()

real_test['lead_time_norm'] = lead_time_norm
real_test['price_norm'] = price_norm

real_test.to_csv('onehot_test.csv', index=False)



In [101]:
real_test.keys()

Index(['Booking_ID', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'meal_type_1',
       'meal_type_2', 'meal_type_3', 'meal_type_4', 'Room_Type_1',
       'Room_Type_2', 'Room_Type_3', 'Room_Type_4', 'Room_Type_5',
       'Room_Type_6', 'Room_Type_7', 'Corporate', 'Complementary', 'Online',
       'Offline', 'Aviation', 'lead_time_norm', 'price_norm'],
      dtype='object')

In [147]:
real_test = pd.read_csv('onehot_test.csv')

feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'required_car_parking_space',
       'arrival_year', 'arrival_month',
       'arrival_date', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'no_of_special_requests', 
       'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
       'Room_Type_1',
       'Room_Type_2', 'Room_Type_3', 'Room_Type_4', 'Room_Type_5',
       'Room_Type_6', 'Room_Type_7', 'Corporate',
       'Complementary', 'Online', 'Offline', 'Aviation',
       'lead_time_norm', 'price_norm']

test = real_test[feature_names]
y_pred = dt_model.predict(test)

sample = pd.DataFrame()
sample['Booking_ID'] = real_test['Booking_ID']
sample['booking_status'] = y_pred
sample.to_csv('sample.csv', index=False)
# real_test.keys()


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Room_Type_1
- Room_Type_2
- Room_Type_3
- Room_Type_4
- Room_Type_5
- ...
Feature names seen at fit time, yet now missing:
- room_type_1
- room_type_2
- room_type_3
- room_type_4
- room_type_5
- ...

  warnings.warn(message, FutureWarning)
